# PyTorch SE-ResNeXt50 to CoreML Conversion

This notebook demonstrates how to convert a trained PyTorch SE-ResNeXt50 model to Apple's CoreML format for deployment on iOS, macOS, and other Apple platforms.

## Overview
- **Source Model**: SE-ResNeXt50 trained for medical image classification
- **Input Model**: `/Users/hc/Documents/PyWorks/best_seresnext50_model.pth`
- **Target Format**: CoreML (.mlmodel)
- **Use Case**: Binary classification (Normal vs Abnormal medical images)

## Conversion Pipeline
1. **Load PyTorch Model**: Restore the trained SE-ResNeXt50 architecture and weights
2. **TorchScript Conversion**: Convert to an intermediate TorchScript format
3. **CoreML Conversion**: Transform TorchScript to CoreML format
4. **Validation**: Test the converted model for correctness
5. **Optimization**: Apply CoreML optimizations for deployment

---

## 1. Import Required Libraries

Import all necessary libraries for PyTorch model loading, TorchScript conversion, and CoreML conversion.

In [5]:
# Core libraries
import os
import sys
import time
import warnings
from typing import Dict, Tuple, Optional

# PyTorch and related libraries
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models.resnet import Bottleneck
from torchvision import transforms

# CoreML conversion
import coremltools as ct
from coremltools.models.neural_network import quantization_utils

# Utilities
import numpy as np
from PIL import Image
import platform

# Display
from datetime import datetime

print("✅ All libraries imported successfully!")
print(f"Conversion started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Platform: {platform.platform()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CoreMLTools version: {ct.__version__}")

# Set device
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

Torch version 2.7.1 has not been tested with coremltools. You may run into unexpected errors. Torch 2.5.0 is the most recent version that has been tested.


✅ All libraries imported successfully!
Conversion started at: 2025-07-03 15:10:35
Platform: macOS-15.3.1-arm64-arm-64bit
PyTorch version: 2.7.1
CoreMLTools version: 8.3.0
Using device: mps


## 2. Load the Trained PyTorch Model

First, we need to recreate the SE-ResNeXt50 model architecture and load the trained weights from the checkpoint file.

In [6]:
# Define the SE (Squeeze-and-Excitation) Layer
class SELayer(nn.Module):
    """Squeeze-and-Excitation layer for channel attention."""
    
    def __init__(self, channel: int, reduction: int = 16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


# Define SE-enhanced Bottleneck block
class SEBottleneck(Bottleneck):
    """ResNeXt Bottleneck block with Squeeze-and-Excitation."""
    
    expansion = 4

    def __init__(self, inplanes: int, planes: int, stride: int = 1, 
                 downsample: Optional[nn.Module] = None, groups: int = 1, 
                 base_width: int = 64, dilation: int = 1, 
                 norm_layer: Optional[nn.Module] = None, se_reduction: int = 16):
        super(SEBottleneck, self).__init__(
            inplanes, planes, stride, downsample, groups, 
            base_width, dilation, norm_layer
        )
        self.se = SELayer(planes * self.expansion, reduction=se_reduction)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        
        # Apply SE attention
        out = self.se(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


def get_seresnext50(num_classes: int = 1, se_reduction: int = 16) -> nn.Module:
    """Create SE-ResNeXt50 model with specified number of classes."""
    
    # Start with pretrained ResNeXt50
    model = models.resnext50_32x4d(pretrained=False)  # We'll load our own weights
    base_width = model.base_width

    def replace_bottlenecks(module: nn.Module, se_reduction_ratio: int, base_width: int) -> None:
        """Recursively replace standard bottlenecks with SE bottlenecks."""
        for name, child_module in module.named_children():
            if isinstance(child_module, Bottleneck):
                # Extract parameters from existing bottleneck
                inplanes = child_module.conv1.in_channels
                planes = child_module.conv3.out_channels // child_module.expansion
                stride = child_module.stride
                downsample = child_module.downsample
                groups = child_module.conv2.groups
                dilation = child_module.conv2.dilation[0]

                # Create new SE bottleneck
                new_bottleneck = SEBottleneck(
                    inplanes=inplanes,
                    planes=planes,
                    stride=stride,
                    downsample=downsample,
                    groups=groups,
                    base_width=base_width,
                    dilation=dilation,
                    se_reduction=se_reduction_ratio
                )

                # Load existing weights (excluding SE layer)
                new_bottleneck.load_state_dict(child_module.state_dict(), strict=False)
                
                # Replace the module
                setattr(module, name, new_bottleneck)
            else:
                # Recursively process child modules
                replace_bottlenecks(child_module, se_reduction_ratio, base_width)

    # Replace all bottlenecks with SE bottlenecks
    replace_bottlenecks(model, se_reduction, base_width)

    # Replace final classifier layer
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)

    return model

# Create the model architecture
print("Creating SE-ResNeXt50 model architecture...")
model = get_seresnext50(num_classes=1, se_reduction=16)
print(f"Model created with {sum(p.numel() for p in model.parameters())} parameters")

Creating SE-ResNeXt50 model architecture...
Model created with 25496897 parameters


In [7]:
# Load the trained model weights
checkpoint_path = '/Users/hc/Documents/PyWorks/best_seresnext50_model.pth'

print(f"Loading checkpoint from: {checkpoint_path}")

# Check if checkpoint file exists
if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"Checkpoint file not found: {checkpoint_path}")

# Load checkpoint
checkpoint = torch.load(checkpoint_path, map_location='cpu')

print("Checkpoint loaded successfully!")
print(f"Training epoch: {checkpoint.get('epoch', 'Unknown')}")
print(f"Validation loss: {checkpoint.get('val_loss', 'Unknown')}")

# Extract model state dict
state_dict = checkpoint['model_state_dict']

# Handle DataParallel models (remove 'module.' prefix if present)
if list(state_dict.keys())[0].startswith('module.'):
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:]  # Remove 'module.' prefix
        new_state_dict[name] = v
    state_dict = new_state_dict
    print("Removed 'module.' prefix from state dict keys")

# Load weights into model
missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)

if missing_keys:
    print(f"⚠️  Missing keys: {missing_keys}")
if unexpected_keys:
    print(f"⚠️  Unexpected keys: {unexpected_keys}")

print("✅ Model weights loaded successfully!")

# Set model to evaluation mode
model.eval()

# Display model configuration
config = checkpoint.get('config', {})
print(f"\nModel Configuration:")
print(f"  • Learning rate: {config.get('learning_rate', 'Unknown')}")
print(f"  • Weight decay: {config.get('weight_decay', 'Unknown')}")
print(f"  • Batch size: {config.get('batch_size', 'Unknown')}")
print(f"  • SE reduction ratio: {config.get('se_reduction_ratio', 16)}")

Loading checkpoint from: /Users/hc/Documents/PyWorks/best_seresnext50_model.pth
Checkpoint loaded successfully!
Training epoch: 11
Validation loss: 0.010342853844721377
Removed 'module.' prefix from state dict keys
✅ Model weights loaded successfully!

Model Configuration:
  • Learning rate: 0.0004
  • Weight decay: 0.0001
  • Batch size: 128
  • SE reduction ratio: 16


## 3. Prepare a Dummy Input for Tracing

Create a dummy input tensor that matches the expected input shape for the model. This will be used for TorchScript tracing.

In [8]:
# Define input specifications
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 1  # For inference, typically batch size of 1
CHANNELS = 3    # RGB images

# ImageNet normalization values (same as used in training)
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

print(f"Input specifications:")
print(f"  • Image size: {IMG_HEIGHT}x{IMG_WIDTH}")
print(f"  • Channels: {CHANNELS}")
print(f"  • Batch size: {BATCH_SIZE}")
print(f"  • Normalization mean: {MEAN}")
print(f"  • Normalization std: {STD}")

# Create dummy input tensor
dummy_input = torch.randn(BATCH_SIZE, CHANNELS, IMG_HEIGHT, IMG_WIDTH)
print(f"\nDummy input shape: {dummy_input.shape}")
print(f"Dummy input dtype: {dummy_input.dtype}")

# Test the model with dummy input to ensure it works
print("\nTesting model with dummy input...")
try:
    with torch.no_grad():
        dummy_output = model(dummy_input)
    print(f"✅ Model test successful!")
    print(f"Output shape: {dummy_output.shape}")
    print(f"Output dtype: {dummy_output.dtype}")
    print(f"Output range: [{dummy_output.min().item():.4f}, {dummy_output.max().item():.4f}]")
    
    # Apply sigmoid to get probability
    probability = torch.sigmoid(dummy_output).item()
    print(f"Sigmoid probability: {probability:.4f}")
    
except Exception as e:
    print(f"❌ Model test failed: {e}")
    raise e

# Define preprocessing transformation (for reference)
preprocess_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

print(f"\nPreprocessing pipeline defined for reference")

Input specifications:
  • Image size: 224x224
  • Channels: 3
  • Batch size: 1
  • Normalization mean: [0.485, 0.456, 0.406]
  • Normalization std: [0.229, 0.224, 0.225]

Dummy input shape: torch.Size([1, 3, 224, 224])
Dummy input dtype: torch.float32

Testing model with dummy input...
✅ Model test successful!
Output shape: torch.Size([1, 1])
Output dtype: torch.float32
Output range: [0.8903, 0.8903]
Sigmoid probability: 0.7089

Preprocessing pipeline defined for reference:


## 4. Convert the PyTorch Model to TorchScript

Convert the PyTorch model to TorchScript format, which serves as an intermediate representation for CoreML conversion.

In [9]:
# Convert PyTorch model to TorchScript using tracing
print("Converting PyTorch model to TorchScript...")
print("=" * 50)

try:
    # Ensure model is in evaluation mode
    model.eval()
    
    # Method 1: Try tracing first (usually works better for most models)
    print("Attempting TorchScript tracing...")
    
    with torch.no_grad():
        traced_model = torch.jit.trace(model, dummy_input)
    
    print("✅ TorchScript tracing successful!")
    
    # Test the traced model
    print("Testing traced model...")
    
    with torch.no_grad():
        traced_output = traced_model(dummy_input)
    
    # Compare outputs to ensure tracing worked correctly
    original_output = model(dummy_input)
    output_diff = torch.abs(traced_output - original_output).max().item()
    
    print(f"✅ Traced model test successful!")
    print(f"Output difference (should be very small): {output_diff:.8f}")
    
    if output_diff > 1e-5:
        print("⚠️  Large output difference detected - may indicate tracing issues")
    
    torchscript_model = traced_model
    
except Exception as trace_error:
    print(f"❌ TorchScript tracing failed: {trace_error}")
    
    # Method 2: Try scripting as fallback
    print("\nAttempting TorchScript scripting as fallback...")
    
    try:
        scripted_model = torch.jit.script(model)
        print("✅ TorchScript scripting successful!")
        
        # Test the scripted model
        with torch.no_grad():
            scripted_output = scripted_model(dummy_input)
        
        print("✅ Scripted model test successful!")
        torchscript_model = scripted_model
        
    except Exception as script_error:
        print(f"❌ TorchScript scripting also failed: {script_error}")
        raise Exception("Both TorchScript tracing and scripting failed")

# Save TorchScript model for reference
torchscript_path = 'seresnext50_torchscript.pt'
torch.jit.save(torchscript_model, torchscript_path)
print(f"\nTorchScript model saved to: {torchscript_path}")

# Get model information
print(f"\nTorchScript Model Information:")
print(f"  • Model type: {type(torchscript_model)}")
print(f"  • Code: {torchscript_model.code if hasattr(torchscript_model, 'code') else 'Not available'}")

# Check model graph
try:
    graph = torchscript_model.graph
    print(f"  • Graph nodes: {len(list(graph.nodes()))}")
    print(f"  • Graph inputs: {len(list(graph.inputs()))}")
    print(f"  • Graph outputs: {len(list(graph.outputs()))}")
except:
    print("  • Graph information not available")

Converting PyTorch model to TorchScript...
Attempting TorchScript tracing...
✅ TorchScript tracing successful!
Testing traced model...
✅ Traced model test successful!
Output difference (should be very small): 0.00000000

💾 TorchScript model saved to: seresnext50_torchscript.pt

TorchScript Model Information:
  • Model type: <class 'torch.jit._trace.TopLevelTracedModule'>
  • Code: def forward(self,
    x: Tensor) -> Tensor:
  fc = self.fc
  avgpool = self.avgpool
  layer4 = self.layer4
  layer3 = self.layer3
  layer2 = self.layer2
  layer1 = self.layer1
  maxpool = self.maxpool
  relu = self.relu
  bn1 = self.bn1
  conv1 = self.conv1
  _0 = (relu).forward((bn1).forward((conv1).forward(x, ), ), )
  _1 = (layer1).forward((maxpool).forward(_0, ), )
  _2 = (layer3).forward((layer2).forward(_1, ), )
  _3 = (avgpool).forward((layer4).forward(_2, ), )
  input = torch.flatten(_3, 1)
  return (fc).forward(input, )

  • Graph nodes: 23
  • Graph inputs: 2
  • Graph outputs: 1


## 5. Convert TorchScript Model to Core ML Format

Use CoreMLTools to convert the TorchScript model to CoreML format with proper input/output specifications.

In [11]:
# Convert TorchScript model to CoreML
print("Converting TorchScript model to CoreML...")
print("=" * 50)

try:
    # Define input specifications for CoreML
    input_shape = ct.Shape(shape=(BATCH_SIZE, CHANNELS, IMG_HEIGHT, IMG_WIDTH))
    
    # Create input type with proper naming and preprocessing
    input_type = [
        ct.TensorType(
            name="image",
            shape=input_shape,
            dtype=np.float32
        )
    ]
    
    # Define output type
    output_type = [
        ct.TensorType(
            name="output", 
            dtype=np.float32
        )
    ]
    
    print(f"Input specifications:")
    print(f"  • Name: 'image'")
    print(f"  • Shape: {input_shape}")
    print(f"  • Type: float32")
    
    # Convert to CoreML
    print(f"\nStarting CoreML conversion...")
    start_time = time.time()
    
    coreml_model = ct.convert(
        torchscript_model,
        inputs=input_type,
        outputs=output_type,
        minimum_deployment_target=ct.target.iOS15,  # For modern Apple devices
        convert_to="mlprogram",  # Use ML Program backend for iOS 15+/macOS 12+
        debug=False
    )
    
    conversion_time = time.time() - start_time
    print(f"✅ CoreML conversion successful!")
    print(f"Conversion time: {conversion_time:.2f} seconds")
    
    # Add model metadata
    print(f"\nAdding model metadata...")
    
    coreml_model.author = "PyTorch to CoreML Converter"
    coreml_model.short_description = "SE-ResNeXt50 for medical image classification"
    coreml_model.version = "1.0"
    
    # Add input/output descriptions
    coreml_model.input_description["image"] = "Input medical image (224x224 RGB)"
    coreml_model.output_description["output"] = "Classification logit (raw output before sigmoid)"
    
    # Display model information
    print(f"\nCoreML Model Information:")
    print(f"  • Model type: {type(coreml_model)}")
    print(f"  • Deployment target: iOS 15+")
    print(f"  • Backend: ML Program")
    
    # Check model inputs and outputs
    spec = coreml_model.get_spec()
    
    print(f"\nModel Specification:")
    print(f"  • Inputs: {len(spec.description.input)}")
    for input_feature in spec.description.input:
        print(f"    - {input_feature.name}: {input_feature.type}")
    
    print(f"  • Outputs: {len(spec.description.output)}")
    for output_feature in spec.description.output:
        print(f"    - {output_feature.name}: {output_feature.type}")
    
    print(f"  • Model size estimate: {len(spec.SerializeToString()) / (1024*1024):.2f} MB")
    
except Exception as e:
    print(f"❌ CoreML conversion failed: {e}")
    print(f"Error type: {type(e)}")
    import traceback
    print(f"Traceback: {traceback.format_exc()}")
    raise e

print(f"\n✅ CoreML model ready for deployment!")

Converting TorchScript model to CoreML...
Input specifications:
  • Name: 'image'
  • Shape: (1, 3, 224, 224)
  • Type: float32

Starting CoreML conversion...


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 235.65 passes/s]



✅ CoreML conversion successful!
Conversion time: 3.17 seconds

Adding model metadata...

CoreML Model Information:
  • Model type: <class 'coremltools.models.model.MLModel'>
  • Deployment target: iOS 15+
  • Backend: ML Program

Model Specification:
  • Inputs: 1
    - image: multiArrayType {
  shape: 1
  shape: 3
  shape: 224
  shape: 224
  dataType: FLOAT32
}

  • Outputs: 1
    - output: multiArrayType {
  shape: 1
  shape: 1
  dataType: FLOAT32
}

  • Model size estimate: 0.12 MB

✅ CoreML model ready for deployment!


## 6. Save the Core ML Model

Save the converted CoreML model to disk with proper naming and optional optimizations.

In [14]:
# Save the CoreML model
print("Saving CoreML model...")
print("=" * 50)

# Define output file path
output_path = "seresnext50_mri_coreml.mlpackage"  # Use .mlpackage for ML Program

try:
    # Save the CoreML model
    print(f"Saving model to: {output_path}")
    coreml_model.save(output_path)
    
    # Get file size
    file_size = os.path.getsize(output_path) / (1024 * 1024)  # Size in MB
    
    print(f"✅ CoreML model saved successfully!")
    print(f"  • File path: {os.path.abspath(output_path)}")
    print(f"  • File size: {file_size:.2f} MB")
    
    # Optional: Create a quantized version for smaller file size
    print(f"\nCreating quantized version for mobile deployment...")
    
    try:
        # Apply 16-bit quantization
        quantized_model = quantization_utils.quantize_weights(coreml_model, nbits=16)
        quantized_path = "seresnext50_medical_classifier_quantized.mlpackage"
        
        quantized_model.save(quantized_path)
        quantized_file_size = os.path.getsize(quantized_path) / (1024 * 1024)
        
        print(f"✅ Quantized model saved!")
        print(f"  • File path: {os.path.abspath(quantized_path)}")
        print(f"  • File size: {quantized_file_size:.2f} MB")
        print(f"  • Size reduction: {((file_size - quantized_file_size) / file_size * 100):.1f}%")
        
    except Exception as quant_error:
        print(f"⚠️  Quantization failed: {quant_error}")
        print("The original model is still available.")
    
    # Create a model with preprocessing built-in (optional)
    print(f"\nCreating model with built-in preprocessing...")
    
    try:
        # Define preprocessing pipeline
        # Note: CoreML preprocessing may differ from PyTorch transforms
        from coremltools.models.neural_network import NeuralNetworkBuilder
        from coremltools.models import datatypes
        
        # This is a simplified version - for production, you might want
        # to include normalization directly in the model
        preprocessing_model = coreml_model  # Use the original model
        
        print(f"✅ Model ready for deployment!")
        print(f"Note: Remember to apply the same preprocessing as during training:")
        print(f"  1. Resize to 224x224")
        print(f"  2. Convert to tensor (0-1 range)")
        print(f"  3. Normalize with mean={MEAN}, std={STD}")
        
    except Exception as prep_error:
        print(f"⚠️  Preprocessing integration skipped: {prep_error}")
    
except Exception as save_error:
    print(f"❌ Failed to save CoreML model: {save_error}")
    raise save_error

# Print deployment information
print(f"\nDeployment Information:")
print(f"  • Compatible with: iOS 15+, macOS 12+, watchOS 8+, tvOS 15+")
print(f"  • Framework: Core ML")
print(f"  • Input: RGB image (224x224)")
print(f"  • Output: Classification logit (apply sigmoid for probability)")
print(f"  • Use case: Stroke MRI Image Classification")

Saving CoreML model...
Saving model to: seresnext50_mri_coreml.mlpackage
✅ CoreML model saved successfully!
  • File path: /Users/hc/Documents/PyWorks/seresnext50_mri_coreml.mlpackage
  • File size: 0.00 MB

Creating quantized version for mobile deployment...
Quantizing using linear quantization
⚠️  Quantization failed: MLModel of type mlProgram cannot be loaded just from the model spec object. It also needs the path to the weights file. Please provide that as well, using the 'weights_dir' argument.
The original model is still available.

Creating model with built-in preprocessing...
✅ Model ready for deployment!
Note: Remember to apply the same preprocessing as during training:
  1. Resize to 224x224
  2. Convert to tensor (0-1 range)
  3. Normalize with mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]

Deployment Information:
  • Compatible with: iOS 15+, macOS 12+, watchOS 8+, tvOS 15+
  • Framework: Core ML
  • Input: RGB image (224x224)
  • Output: Classification logit (apply

## 7. Test Core ML Model Loading

Verify that the saved CoreML model can be loaded correctly and validate its functionality.

In [15]:
# Test CoreML model loading and inference
print("Testing CoreML model loading and inference...")
print("=" * 50)

try:
    # Load the saved CoreML model
    print("Loading CoreML model from disk...")
    loaded_coreml_model = ct.models.MLModel(output_path)
    
    print("✅ CoreML model loaded successfully!")
    
    # Display model metadata
    spec = loaded_coreml_model.get_spec()
    
    print(f"\nModel Metadata:")
    print(f"  • Author: {loaded_coreml_model.author}")
    print(f"  • Description: {loaded_coreml_model.short_description}")
    print(f"  • Version: {loaded_coreml_model.version}")
    
    # Display input/output information
    print(f"\nInput/Output Information:")
    for input_feature in spec.description.input:
        print(f"  • Input '{input_feature.name}':")
        if input_feature.type.HasField('multiArrayType'):
            shape = input_feature.type.multiArrayType.shape
            print(f"    - Shape: {list(shape)}")
            print(f"    - Data type: {input_feature.type.multiArrayType.dataType}")
    
    for output_feature in spec.description.output:
        print(f"  • Output '{output_feature.name}':")
        if output_feature.type.HasField('multiArrayType'):
            shape = output_feature.type.multiArrayType.shape
            print(f"    - Shape: {list(shape)}")
            print(f"    - Data type: {output_feature.type.multiArrayType.dataType}")
    
    # Test inference with the loaded model
    print(f"\nTesting inference with loaded CoreML model...")
    
    # Prepare test input (numpy array)
    test_input = dummy_input.numpy()
    input_dict = {"image": test_input}
    
    # Run inference
    start_time = time.time()
    coreml_output = loaded_coreml_model.predict(input_dict)
    inference_time = time.time() - start_time
    
    print(f"✅ CoreML inference successful!")
    print(f"Inference time: {inference_time*1000:.2f} ms")
    
    # Extract output
    coreml_result = coreml_output["output"]
    print(f"Output shape: {coreml_result.shape}")
    print(f"Output value: {coreml_result}")
    
    # Apply sigmoid to get probability
    coreml_probability = 1 / (1 + np.exp(-coreml_result))
    print(f"Sigmoid probability: {coreml_probability}")
    
    # Compare with original PyTorch model
    print(f"\nComparing with PyTorch model...")
    
    with torch.no_grad():
        pytorch_output = model(dummy_input)
        pytorch_probability = torch.sigmoid(pytorch_output)
    
    # Calculate differences
    output_diff = np.abs(coreml_result - pytorch_output.numpy())
    prob_diff = np.abs(coreml_probability - pytorch_probability.numpy())
    
    print(f"Output difference: {output_diff.max():.8f}")
    print(f"Probability difference: {prob_diff.max():.8f}")
    
    if output_diff.max() < 1e-4:
        print("✅ CoreML and PyTorch outputs match closely!")
    else:
        print("⚠️  Significant difference detected between CoreML and PyTorch outputs")
    
    # Performance benchmark
    print(f"\nPerformance Benchmark (100 inferences):")
    
    # PyTorch benchmark
    pytorch_times = []
    model.eval()
    for _ in range(100):
        start_time = time.time()
        with torch.no_grad():
            _ = model(dummy_input)
        pytorch_times.append(time.time() - start_time)
    
    avg_pytorch_time = np.mean(pytorch_times) * 1000  # Convert to ms
    
    # CoreML benchmark
    coreml_times = []
    for _ in range(100):
        start_time = time.time()
        _ = loaded_coreml_model.predict(input_dict)
        coreml_times.append(time.time() - start_time)
    
    avg_coreml_time = np.mean(coreml_times) * 1000  # Convert to ms
    
    print(f"  • PyTorch average: {avg_pytorch_time:.2f} ms")
    print(f"  • CoreML average: {avg_coreml_time:.2f} ms")
    
    if avg_coreml_time < avg_pytorch_time:
        speedup = avg_pytorch_time / avg_coreml_time
        print(f"  • CoreML is {speedup:.2f}x faster!")
    else:
        slowdown = avg_coreml_time / avg_pytorch_time
        print(f"  • CoreML is {slowdown:.2f}x slower")
    
except Exception as test_error:
    print(f"❌ CoreML model testing failed: {test_error}")
    import traceback
    print(f"Traceback: {traceback.format_exc()}")
    raise test_error

print(f"\n🎉 CoreML conversion and testing completed successfully!")

Testing CoreML model loading and inference...
Loading CoreML model from disk...
✅ CoreML model loaded successfully!

Model Metadata:
  • Author: PyTorch to CoreML Converter
  • Description: SE-ResNeXt50 for medical image classification
  • Version: 1.0

Input/Output Information:
  • Input 'image':
    - Shape: [1, 3, 224, 224]
    - Data type: 65568
  • Output 'output':
    - Shape: [1, 1]
    - Data type: 65568

Testing inference with loaded CoreML model...
✅ CoreML inference successful!
Inference time: 31.98 ms
Output shape: (1, 1)
Output value: [[0.89990234]]
Sigmoid probability: [[0.71092945]]

Comparing with PyTorch model...
Output difference: 0.00963944
Probability difference: 0.00198501
⚠️  Significant difference detected between CoreML and PyTorch outputs

Performance Benchmark (100 inferences):
✅ CoreML model loaded successfully!

Model Metadata:
  • Author: PyTorch to CoreML Converter
  • Description: SE-ResNeXt50 for medical image classification
  • Version: 1.0

Input/Outpu